In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure, color, io

In [5]:
img1 = cv2.imread("Spherical-Aluminum-Nitride.png")
img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

In [6]:
pixels_to_um = 0.5 # 1 pixel = 500 nm (got this from the metadata of original image)


In [24]:
ret1, thresh = cv2.threshold(img, 30, 255, cv2.THRESH_BINARY)
# ret1, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imshow('Colored Grains', thresh)
cv2.waitKey(0)

-1

In [25]:
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
cv2.imshow('Colored Grains',opening )
cv2.waitKey(0)


-1

In [ ]:
# from skimage.segmentation import clear_border
# opening = clear_border(opening) 

In [26]:
sure_bg = cv2.dilate(opening,kernel,iterations=2)
cv2.imshow('Colored Grains',sure_bg )
cv2.waitKey(0)

-1

In [45]:
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,3)
cv2.imshow('Colored Grains',dist_transform)
cv2.waitKey(0)

-1

In [28]:
ret2, sure_fg = cv2.threshold(dist_transform,0.2*dist_transform.max(),255,0)

In [29]:
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)
cv2.imshow('Colored Grains', unknown)
cv2.waitKey(0)

-1

In [30]:
ret3, markers = cv2.connectedComponents(sure_fg)


In [31]:
markers = markers+10

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

In [32]:
markers = cv2.watershed(img1,markers)

In [41]:
img1[markers == -1] = [0,255,255]  

img2 = color.label2rgb(markers, bg_label=0)

cv2.imshow('Overlay on original image', img1)
cv2.imshow('Colored Grains', img2)
cv2.waitKey(0)


-1

In [34]:
regions = measure.regionprops(markers, intensity_image=img)


In [38]:
#Can print various parameters for all objects
for prop in regions:
    print('Label: {} Area: {}'.format(prop.label, prop.area))

#Best way is to output all properties to a csv file
#Let us pick which ones we want to export. 

# propList = ['Area',
            # 'equivalent_diameter', #Added... verify if it works
            # 'orientation', #Added, verify if it works. Angle btwn x-axis and major axis.
            # 'MajorAxisLength',
            # 'MinorAxisLength',
            # 'Perimeter',
            # 'MinIntensity',
            # 'MeanIntensity',
            # 'MaxIntensity']    
    


Label: 10 Area: 35842.0
Label: 11 Area: 4927.0
Label: 12 Area: 4231.0
Label: 13 Area: 861.0
Label: 14 Area: 3209.0
Label: 15 Area: 556.0
Label: 16 Area: 4832.0
Label: 17 Area: 8487.0
Label: 18 Area: 8127.0
Label: 19 Area: 7401.0
Label: 20 Area: 218.0
Label: 21 Area: 6328.0
Label: 22 Area: 338.0
Label: 23 Area: 857.0
Label: 24 Area: 8924.0
Label: 25 Area: 3498.0
Label: 26 Area: 8486.0
Label: 27 Area: 802.0
Label: 28 Area: 6308.0
Label: 29 Area: 22.0
Label: 30 Area: 1.0
Label: 31 Area: 9483.0
Label: 32 Area: 2442.0
Label: 33 Area: 2364.0
Label: 34 Area: 890.0
Label: 35 Area: 1383.0
Label: 36 Area: 656.0


In [42]:
propList = ['Area',
            'equivalent_diameter', #Added... verify if it works
            'orientation', #Added, verify if it works. Angle btwn x-axis and major axis.
            'MajorAxisLength',
            'MinorAxisLength',
            'Perimeter',
            'MinIntensity',
            'MeanIntensity',
            'MaxIntensity']    
    

In [44]:
output_file = open('image_measurements.csv', 'w')
output_file.write('Grain #' + "," + "," + ",".join(propList) + '\n') #join strings in array by commas, 
#First cell to print grain number
#Second cell blank as we will not print Label column

grain_number = 1
for region_props in regions:
    output_file.write(str(grain_number) + ',')
    #output cluster properties to the excel file
#    output_file.write(str(region_props['Label']))
    for i,prop in enumerate(propList):
        if(prop == 'Area'): 
            to_print = region_props[prop]*pixels_to_um**2   #Convert pixel square to um square
        elif(prop == 'orientation'): 
            to_print = region_props[prop]*57.2958  #Convert to degrees from radians
        elif(prop.find('Intensity') < 0):          # Any prop without Intensity in its name
            to_print = region_props[prop]*pixels_to_um
        else: 
            to_print = region_props[prop]     #Reamining props, basically the ones with Intensity in its name
        output_file.write(',' + str(to_print))
    output_file.write('\n')
    grain_number += 1
    
output_file.close()   #Closes the file, otherwise it would be read only. 